COMBINE sammanställningsresultat för 2020-2024

In [ ]:
years = range(2022, 2024)
files = [f"data/yh-sokande-{year}-behorighet-reell-kompetens.xlsx" for year in years]

df_list = []

for file in files:
    year = file.split('-')[-1].split('.')[0]  # Extract year from filename
    try:
        df = pd.read_excel(file, sheet_name='Sökande 2024 - Tab1', header=5)
        df['År'] = int(year)  # Add year column
        df_list.append(df)
    except Exception as e:
        print(f"Error processing {file}: {e}")

# Combine all data
combined_df = pd.concat(df_list, ignore_index=True)

# Save to new Excel file
combined_df.to_excel("sammanstallning-resultat-2023-2024.xlsx", index=False)

In [ ]:
import pandas as pd

df1 = pd.read_excel("data/sammanstallning-resultat-2022-2024.xlsx")
df2 = pd.read_excel("data/sammanstallning-resultat-2023-2024.xlsx")

# Combine the data
combined_df = pd.concat([df1, df2], ignore_index=True)

combined_df.to_excel("sammanstallning-resultat-2020-2024-updated.xlsx", index=False)

Combined file saved as 'sammanstallning-resultat-2020-2024-updated.xlsx'


## Program

In [5]:
df = pd.read_excel("data/sammanstallning-resultat-2020-2024-updated.xlsx")
df.head()

,Utbildningsområde,Utbildningsnamn,Län,Kommun,Flera studiekommuner,Antal kommuner,Antal län,YH-poäng,Utbildningsanordnare administrativ enhet,Diarienummer,År,Flera kommuner
0,Data/IT,.NET Utvecklare,Västra Götaland,Göteborg,Nej,1,1.0,400,Plushögskolan AB - Teknikhögskolan,MYH 2020/4403,2020,NaN
1,Data/IT,.NET-utvecklare,Stockholm,Stockholm,Nej,1,1.0,400,IT-Högskolan Stockholm AB,MYH 2020/5766,2020,NaN
2,Data/IT,.NET-utvecklare,Västra Götaland,Göteborg,Nej,1,1.0,400,IT-Högskolan Sverige AB,MYH 2020/5764,2020,NaN
3,Data/IT,Agil Javautvecklare,Gävleborg,Gävle,Nej,1,1.0,400,Edugrade AB,MYH 2020/5827,2020,NaN
4,Data/IT,Agil projektledare,Skåne,Malmö,Nej,1,1.0,225,IHM Business School AB Malmö,MYH 2020/5354,2020,NaN
